## 가위바위보_구별기
-----

### 반복 사용되는 모듈 임포트 

In [9]:
from PIL import Image
import matplotlib.pyplot as plt
import os, glob, cv2

print("PIL 라이브러리 import 완료!")

PIL 라이브러리 import 완료!


### 가위, 바위, 보 이미지 전처리

In [103]:
def resize_images(img_path): 
    for folder in os.listdir(img_path):
        if folder not in ['rock', 'scissor', 'paper']: 
            continue   
        images=glob.glob(img_path + f"/{folder}/*.jpg")  
        print(f"{len(images)} {folder} images to be resized.")
        # 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.
        target_size=(28,28)
        for img in images:
            old_img=cv2.imread(img,cv2.IMREAD_GRAYSCALE)
            new_img=cv2.resize(old_img,target_size,Image.ANTIALIAS)
            cv2.imwrite(img,new_img)
        print(f"{len(images)} {folder} images resized.")

In [104]:
folder_name = input("test/train 중 하나를 입력하세요")
image_dir_path = os.getenv("HOME") + f"/aiffel/rock_scissor_paper/{folder_name}"
resize_images(image_dir_path)

print(f"{folder_name} 이미지 resize 완료!")

test/train 중 하나를 입력하세요test
92 paper images to be resized.
92 paper images resized.
77 rock images to be resized.
77 rock images resized.
66 scissor images to be resized.
66 scissor images resized.
test 이미지 resize 완료!


### 전처리된 이미지 행렬화 및 정규화

In [134]:
import numpy as np

def load_data(img_path, number_of_data=300):
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=28
    color=1
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size, color)
    labels=np.zeros(number_of_data,dtype=np.int32)
     
    # number_of_data를 3분의 1로 나눠 가위/바위/보를 균등하게 채움   
    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        if idx > (number_of_data//3)-1: # 가위 이미지의 개수상한 초과시 추가하지 않음.
            continue
        img = np.array(Image.open(file),dtype=np.uint8).reshape(28,28,1)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1
        
    for file in glob.iglob(img_path+'/rock/*.jpg'):
        if idx > (number_of_data//3*2)-1: # 바위 이미지의 개수상한 초과시 추가하지 않음.
            continue
        img = np.array(Image.open(file),dtype=np.uint8).reshape(28,28,1)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  

    for file in glob.iglob(img_path+'/paper/*.jpg'):
        if idx > number_of_data-1:  # 보 이미지의 개수상한 초과시 추가하지 않음.
            continue
        img = np.array(Image.open(file),dtype=np.uint8).reshape(28,28,1)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print(f"처리된 데이터의 이미지 개수는 {idx}개 입니다.")
    return imgs, labels

In [135]:
folder_name = input("train/test 중 하나를 입력하세요")
image_dir_path = os.getenv("HOME") + f"/aiffel/rock_scissor_paper/{folder_name}"
if folder_name == "train":
    (x_train, y_train)=load_data(image_dir_path,3300)
    x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화
    x_train_reshaped=x_train_norm.reshape( -1, 28, 28, 1)  # 정규화된 이미지 reshape
    print(f"x_train shape: {x_train_reshaped.shape}")
    print(f"y_train shape: {y_train.shape}")
else: 
    (x_test, y_test)=load_data(image_dir_path)
    x_test_norm = x_test/255.0   # 입력은 0~1 사이의 값으로 정규화
    x_train_reshaped=x_train_norm.reshape( -1, 28, 28, 1)  # 정규화된 이미지 reshape
    x_test_reshaped=x_test_norm.reshape( -1, 28, 28, 1)
    print(f"x_test reshaped: {x_test_reshaped.shape}")
    print(f"y_test reshaped: {y_test.shape}")

train/test 중 하나를 입력하세요train
처리된 데이터의 이미지 개수는 3207개 입니다.
x_train shape: (3300, 28, 28, 1)
y_train shape: (3300,)


### 학습 모델만들기  

[컨볼루션 신경망 레이어 이야기](https://tykimos.github.io/2017/01/27/CNN_Layer_Talk/)를 참조하여 실습을 진행했다

In [159]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, MaxPooling2D, Flatten, Dense
import numpy as np

model = Sequential()
#4X4크기의 필터 16개를 쌓고, 입력 이미지의 크기를 28X28X1로 지정한다.  
model.add(Conv2D(16, (4,4), activation='relu', input_shape=(28,28,1)))
model.add(MaxPool2D(2,2))
model.add(Conv2D(16, (4,4), activation='relu', input_shape=(28,28,1)))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
# 입력뉴런 4개 출력뉴런 128개인 Dense레이어를 model에 추가           
model.add(Dense(128, activation='relu', input_dim=4))
model.add(Dense(16, activation='softmax'))

model.summary()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_42 (Conv2D)           (None, 25, 25, 16)        272       
_________________________________________________________________
max_pooling2d_42 (MaxPooling (None, 12, 12, 16)        0         
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 9, 9, 16)          4112      
_________________________________________________________________
max_pooling2d_43 (MaxPooling (None, 4, 4, 16)          0         
_________________________________________________________________
flatten_20 (Flatten)         (None, 256)               0         
_________________________________________________________________
dense_39 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_40 (Dense)             (None, 16)              

### 학습

In [160]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

# 모델이 20번 학습한다.                                      
model.fit(x_train_reshaped, y_train, epochs=20)

Epoch 1/20
104/104 [==============================] - 1s 3ms/step - loss: 1.5631 - accuracy: 0.3415
Epoch 2/20
104/104 [==============================] - 0s 3ms/step - loss: 1.0140 - accuracy: 0.4451
Epoch 3/20
104/104 [==============================] - 0s 3ms/step - loss: 0.8783 - accuracy: 0.5644
Epoch 4/20
104/104 [==============================] - 0s 3ms/step - loss: 0.7846 - accuracy: 0.6363
Epoch 5/20
104/104 [==============================] - 0s 3ms/step - loss: 0.6583 - accuracy: 0.7209
Epoch 6/20
104/104 [==============================] - 0s 3ms/step - loss: 0.5910 - accuracy: 0.7514
Epoch 7/20
104/104 [==============================] - 0s 3ms/step - loss: 0.4934 - accuracy: 0.8163
Epoch 8/20
104/104 [==============================] - 0s 3ms/step - loss: 0.4476 - accuracy: 0.8320
Epoch 9/20
104/104 [==============================] - 0s 3ms/step - loss: 0.4274 - accuracy: 0.8322
Epoch 10/20
104/104 [==============================] - 0s 3ms/step - loss: 0.3882 - accuracy: 0.8534

### 테스트

In [161]:
test_loss, test_accuracy = model.evaluate(x_test_reshaped, y_test, verbose=2)
print(f'test_loss: {test_loss}')
print(f'test_accuracy: {test_accuracy}')

10/10 - 0s - loss: 2.1581 - accuracy: 0.6433
test_loss: 2.158133029937744
test_accuracy: 0.6433333158493042


## 후기
-----
이번 과제를 수행하는 동안 어려웠던 점은 역시 **accuracy 60을 넘기는 것**이었다.

도중에 여러번 60을 넘긴 경우가 있었지만, 참지못하고 다시 시도하는 바람에 다시 60을 만들기까지 긴 시간이 걸렸다.

하지만 그러면서 여러가지 시도를 하다보니 배우게 된 부분 역시 많았다.

<br>

1. cv2 모듈을 활용한 이미지파일 그레이스케일 처리

2. 인공신경망의 


